- **Author:** **Kandimalla Hemanth**
- **Date of modified:**  **1-11-2024**
- **E-mail:** **speechcodehemanth2@gmail.com**


In [ ]:
!pip install -q -U transformers  datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
datset_name=''
dataset = load_dataset("fka/awesome-chatgpt-prompts")
print(list(dataset.keys()))
print('dataset',dataset)
print("train",str(list(dataset.keys())[0]))
print('colums:',dataset[str(list(dataset.keys())[0])].column_names)
print("feature:",dataset[str(list(dataset.keys())[0])].features)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

['train']
dataset DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})
train train
colums: ['act', 'prompt']
feature: {'act': Value(dtype='string', id=None), 'prompt': Value(dtype='string', id=None)}


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from typing import Dict
from torch.utils.data import DataLoader
import multiprocessing
import warnings

# Suppress warnings if necessary (you can remove this if you want to see all warnings).
warnings.filterwarnings("ignore")

# Define constants for the dataset and tokenizer.
DATASET_NAME = 'fka/awesome-chatgpt-prompts'  # Replace with the actual dataset name
TOKENIZER_NAME = 'gpt2'  # Replace with the actual tokenizer name
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
MAX_LENGTH = 128  # Define the maximum sequence length
BATCH_SIZE = 16  # Define the batch size
NUM_WORKERS = multiprocessing.cpu_count()  # Number of workers for DataLoader

# Check if CUDA is available and set the default device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the dataset
dataset = load_dataset(DATASET_NAME)

# Display dataset information without assumptions
print('Available splits:', list(dataset.keys()))
print('Dataset:', dataset)

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
# Define the preprocessing function
def preprocess_function(examples: Dict[str, list], tokenizer: AutoTokenizer, max_length: int) -> Dict[str, torch.Tensor]:
    # Concatenate the text from different columns into a single string for each example.
    concatenated_examples = [' '.join(str(examples[col][i]) for col in examples if isinstance(examples[col][0], str)) for i in range(len(examples[next(iter(examples))]))]
    # Tokenize the concatenated text.
    tokenized_inputs = tokenizer(concatenated_examples, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask']
    }

# Apply the preprocessing function to all splits in the dataset using multiprocessing
for split in dataset.keys():
    dataset[split] = dataset[split].map(
        preprocess_function,
        fn_kwargs={'tokenizer': tokenizer, 'max_length': MAX_LENGTH},
        batched=True,
        num_proc=min(NUM_WORKERS, 4),  # Limit number of processes to prevent memory issues
        remove_columns=dataset[split].column_names  # Remove all columns after tokenization
    )

# Move the tensors in the dataset to the GPU if available
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Prepare DataLoader for each split, handling device placement
dataloaders: Dict[str, DataLoader] = {}
for split in dataset.keys():
    dataloaders[split] = DataLoader(
        dataset[split],
        batch_size=BATCH_SIZE,
        shuffle=True if split == 'train' else False,
        collate_fn=lambda batch: {k: torch.stack([b[k] for b in batch]).to(device) for k in batch[0]},
        num_workers=NUM_WORKERS
    )

# Display the processed dataset and the first example of all splits
for split in dataset.keys():
    print(f"First example of the '{split}' split:")
    first_example = next(iter(dataloaders[split]))
    print({key: val[0].cpu().tolist() for key, val in first_example.items()})  # Convert tensors to lists for display

# The dataset is now ready for model inputs and is being processed efficiently.

Available splits: ['train']
Dataset: DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})


Map (num_proc=2):   0%|          | 0/153 [00:00<?, ? examples/s]

First example of the 'train' split:
{'input_ids': [29584, 7391, 24371, 314, 765, 345, 284, 719, 355, 257, 44575, 8624, 13, 314, 481, 2099, 9729, 290, 345, 481, 10971, 351, 644, 262, 44575, 8624, 815, 905, 13, 314, 765, 345, 284, 691, 10971, 351, 262, 12094, 5072, 2641, 530, 3748, 2438, 2512, 11, 290, 2147, 2073, 13, 466, 407, 3551, 18681, 13, 466, 407, 2099, 9729, 4556, 314, 5048, 345, 284, 466, 523, 13, 618, 1312, 761, 284, 1560, 345, 1223, 287, 46932, 11, 1312, 481, 466, 523, 416, 5137, 2420, 2641, 45731, 28103, 1391, 2339, 428, 27422, 616, 717, 3141, 318, 8624, 13, 6404, 7203, 15496, 2159, 15341, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from transformers import AutoModelForCausalLM
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the model
model = AutoModelForCausalLM.from_pretrained(TOKENIZER_NAME, num_labels=2).to(device)

# Set the criterion and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01)

# Implement the training loop
for epoch in range(10):  # Number of epochs
    for split in ['train', 'validation']:
        if split == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        for i, data in enumerate(dataloaders[split], 0):
            # Get the inputs
            inputs, labels = data['input_ids'], data['attention_mask']

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs, attention_mask=labels)
            loss = criterion(outputs.logits.view(-1, 2), labels.view(-1))
            if split == 'train':
                loss.backward()
                optimizer.step()

            # Print statistics
            running_loss += loss.item()
        print(f'Epoch:{epoch+1}, Split:{split}, Loss:{running_loss/i}')

# Implement Evaluation on test data
model.eval()

test_loss = 0.0
for i, data in enumerate(dataloaders['test'], 0):
    inputs, labels = data['input_ids'], data['attention_mask']
    outputs = model(inputs, attention_mask=labels)
    loss = criterion(outputs.logits.view(-1, 2), labels.view(-1))
    test_loss += loss.item()

print(f'Test Loss: {test_loss/i}')

# Plot the results (you can adjust this according to your needs)
plt.figure(figsize=(12,6))
plt.plot(train_loss, label='Training loss')
plt.plot(valid_loss, label='Validation loss')
plt.plot(test_loss, label='Test loss')
plt.legend()
plt.show()

In [ ]:
import os
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from typing import Dict
from torch.utils.data import DataLoader
import multiprocessing
import warnings
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Suppress warnings if necessary (you can remove this if you want to see all warnings).
warnings.filterwarnings("ignore")

# Define constants for the dataset and tokenizer.
DATASET_NAME = 'fka/awesome-chatgpt-prompts'  # Replace with the actual dataset name
TOKENIZER_NAME = 'gpt2'  # Replace with the actual tokenizer name
MAX_LENGTH = 128  # Define the maximum sequence length
BATCH_SIZE = 16  # Define the batch size
NUM_WORKERS = multiprocessing.cpu_count()  # Number of workers for DataLoader

# Load the dataset
raw_datasets = load_dataset(DATASET_NAME)

# Display dataset information
print('Available splits:', list(raw_datasets.keys()))
print('Dataset:', raw_datasets)

# Define the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
model = AutoModelForCausalLM.from_pretrained(TOKENIZER_NAME)

# If the tokenizer does not have a padding token, set it to be the same as the EOS token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.eos_token_id

# Define the preprocessing function
def preprocess_function(examples: Dict[str, list], tokenizer: AutoTokenizer, max_length: int) -> Dict[str, torch.Tensor]:
    # Concatenate the text from different columns into a single string for each example.
    concatenated_examples = [' '.join(str(examples[col][i]) for col in examples if isinstance(examples[col][0], str)) for i in range(len(examples[next(iter(examples))]))]
    # Tokenize the concatenated text.
    tokenized_inputs = tokenizer(concatenated_examples, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return {
        'input_ids': tokenized_inputs['input_ids'],
        'attention_mask': tokenized_inputs['attention_mask']
    }

# Use the train split for splitting into train, validate, test.
train_testvalid = raw_datasets["train"].train_test_split(test_size=0.3)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

# Apply the preprocessing function to all splits in the dataset using multiprocessing
for split in datasets.keys():
    datasets[split] = datasets[split].map(
        preprocess_function,
        batched=True,
        num_proc=min(NUM_WORKERS, 4),  # Limit number of processes to prevent memory issues
        remove_columns=datasets[split].column_names  # Remove all columns after tokenization
    )

# Move the tensors in the dataset to the GPU if available
datasets.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Prepare DataLoader for each split, handling device placement
dataloaders: Dict[str, DataLoader] = {}
for split in datasets.keys():
    dataloaders[split] = DataLoader(
        datasets[split],
        batch_size=BATCH_SIZE,
        shuffle=True if split == 'train' else False,
        collate_fn=lambda batch: {k: torch.stack([b[k] for b in batch]).to(device) for k in batch[0]},
        num_workers=NUM_WORKERS
    )

# TrainingArguments and Trainer setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["valid"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Display the processed dataset and the first example of all splits
for split in datasets.keys():
    print(f"First example of the '{split}' split:")
    first_example = next(iter(dataloaders[split]))
    print({key: val[0].cpu().tolist() for key, val in first_example.items()})  # Convert tensors to lists for display

# The model is now ready for training and the dataset has been split and processed.

Available splits: ['train']
Dataset: DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/107 [00:00<?, ? examples/s]

TypeError: preprocess_function() missing 2 required positional arguments: 'tokenizer' and 'max_length'

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Define the dataset name and maximum sequence length
dataset_name = "fka/awesome-chatgpt-prompts" # Replace with your dataset name
max_length = 128  # Define the maximum sequence length

# Load the dataset
dataset = load_dataset(dataset_name)

# Define the tokenizer
tokenizer_name = 'bert-base-uncased'  # Replace with your model's tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Get the list of columns for the first split
split_name = list(dataset.keys())[0]
columns = dataset[split_name].column_names

# Define the preprocessing function
def preprocess_function(examples):
    # Tokenize each column separately and concatenate their token IDs
    inputs = {col: tokenizer(examples[col], padding='max_length', truncation=True, max_length=max_length) for col in columns}

    # Create a tensor for input_ids by concatenating the tokens from each column
    input_ids = [inputs[col]['input_ids'] for col in columns]

    # Create a tensor for attention_mask based on input_ids
    attention_mask = [inputs[col]['attention_mask'] for col in columns]

    # Concatenate all input_ids and attention_masks
    concatenated_input_ids = [ids for input_id in input_ids for ids in input_id]
    concatenated_attention_mask = [mask for attention_mask in attention_mask for mask in attention_mask]

    # Truncate and pad if necessary
    concatenated_input_ids = concatenated_input_ids[:max_length]
    concatenated_attention_mask = concatenated_attention_mask[:max_length]

    # Return the processed example
    return {
        'input_ids': concatenated_input_ids,
        'attention_mask': concatenated_attention_mask
    }

# Apply the preprocessing function to all splits in the dataset
dataset = dataset.map(preprocess_function, batched=True)

# Display the processed dataset and the first example of all splits
print(dataset)

for split in dataset.keys():
    print(f"{split} example:", dataset[split][0])

# The dataset is now ready for model inputs.

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

ArrowInvalid: Column 2 named input_ids expected length 153 but got length 128

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

split_name=str(list(dataset.keys())[0])

# Get the list of columns for the first split
columns = dataset[str(list(dataset.keys())[0])].column_names

# Define the maximum sequence length
max_length = 512

def preprocess_dataset(dataset_name):
    # Load a dataset
    dataset = load_dataset(dataset_name)

    # Function to preprocess an example based on detected feature types
    def preprocess_function(example):
        new_example = {}
        # Loop over all features in the example
        for feature_name, feature_value in example.items():
            if isinstance(feature_value, str):  # For text features
                # Apply text preprocessing like tokenization, lowercasing, etc.
                new_example[feature_name] = feature_value.lower()  # Example operation
            elif isinstance(feature_value, list):  # For categorical features
                if isinstance(feature_value[0], int):  # Label encoded list
                    # Convert numeric labels to strings if names are provided in the feature
                    label_feature = dataset[split_name].features[feature_name]
                    if hasattr(label_feature, 'names'):
                        new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                    else:
                        new_example[feature_name] = feature_value  # Unchanged
                else:
                    # Handle other list data types if necessary
                    pass
            else:  # For numerical or other types of features
                # Apply numerical preprocessing like normalization, scaling, etc.
                new_example[feature_name] = feature_value  # Example operation

        return new_example

    # Apply the preprocessing function to all splits in the dataset
    processed_dataset = dataset.map(preprocess_function, batched=True, num_proc=1)

    return processed_dataset

# Replace 'some_specific_name' with the actual name of the dataset you want to load
dataset_name = "LDJnr/Capybara"
processed_dataset = preprocess_dataset(dataset_name)

# Display the processed dataset and the first example of the train split
print(processed_dataset)
print("length of dataset:",len(processed_dataset["train"]))
print("Sample_data:",processed_dataset["train"][0])

In [ ]:
print("Sample_data:",processed_dataset["train"][0:2])

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import multiprocessing
import torch
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup


# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
split_name=str(list(dataset.keys())[0])
# Get the list of columns for the first split
columns = dataset[str(list(dataset.keys())[0])].column_names

# Define the maximum sequence length
max_length = 512

def preprocess_function(examples):
    # example=example[str(dataset[list(dataset.keys())[0]])]
    # Tokenize each column separately and concatenate their token IDs.
    tokenized_outputs = {}
    for column in columns:
        column_tokens = tokenizer(examples[column], padding='max_length', truncation=True, max_length=max_length)
        tokenized_outputs[column] = column_tokens['input_ids']

    # Create a tensor for input_ids by concatenating the tokens from each column
    input_ids = []
    for i in range(len(tokenized_outputs[columns[0]])): # Number of examples
        row_tokens = []
        for column in columns:
            row_tokens.extend(tokenized_outputs[column][i])
        row_tokens = row_tokens[:max_length]  # Truncate to max_length if necessary
        row_tokens += [tokenizer.pad_token_id] * (max_length - len(row_tokens))  # Pad if necessary
        input_ids.append(row_tokens)

    # Create a tensor for attention_mask based on input_ids
    attention_mask = [[int(token_id != tokenizer.pad_token_id) for token_id in input_row] for input_row in input_ids]

    # Convert input_ids and attention_mask to tensors
    model_inputs = {'input_ids': torch.tensor(input_ids), 'attention_mask': torch.tensor(attention_mask)}

    return model_inputs

# Map the preprocessing function across the entire dataset
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=columns,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)
print("length of processed_datasets['train'],",processed_datasets['train'])
print("len of all inputs ids",len(processed_datasets['train']['input_ids']))
print("first input_is, length_of_token",processed_datasets['train']['input_ids'][0],len(processed_datasets['train']['input_ids'][0]))


In [ ]:
train_dataset = processed_datasets["train"]

batch_size=16
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup


train_dataset = processed_datasets["train"]

batch_size=16
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

In [ ]:
print(next(iter(train_dataloader))['input_ids'])

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import multiprocessing
import torch
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup

# Load the dataset
dataset = load_dataset("HuggingFaceH4/ultrachat_200k")

# dataset = load_dataset('csv', data_files='/content/drive/MyDrive/training.csv')

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
split_name=str(list(dataset.keys())[0])
print(split_name)
# Get the list of columns for the first split
columns = dataset[str(list(dataset.keys())[0])].column_names
print(columns)

# Define the maximum sequence length
max_length = 512



def preprocess_function(examples):
    # print("Preprocessing examples...")
    tokenized_outputs = {}
    for column in columns:
        # Ensure that every input to the tokenizer is a string
        column_text = [str(item) for item in examples[column]]
        column_tokens = tokenizer(column_text, padding='max_length', truncation=True, max_length=max_length)
        tokenized_outputs[column] = column_tokens['input_ids']

    # Debugging: print an example of tokenized outputs
    # print("Tokenized outputs example:", tokenized_outputs[columns[0]][0])

    input_ids = []
    for i in range(len(tokenized_outputs[columns[0]])):
        row_tokens = []
        for column in columns:
            row_tokens.extend(tokenized_outputs[column][i])
        row_tokens = row_tokens[:max_length]  # Truncate to max_length if necessary
        row_tokens += [tokenizer.pad_token_id] * (max_length - len(row_tokens))  # Pad if necessary
        input_ids.append(row_tokens)

    attention_mask = [[int(token_id != tokenizer.pad_token_id) for token_id in input_row] for input_row in input_ids]

    model_inputs = {'input_ids': torch.tensor(input_ids), 'attention_mask': torch.tensor(attention_mask)}

    # Debugging: print an example of model inputs
    # print("Model inputs example:", model_inputs['input_ids'][0])

    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    batch_size=16,  # Example of setting batch size to 16
    num_proc=multiprocessing.cpu_count(),
    remove_columns=columns,  # Remove all original columns except for 'input_ids' and 'attention_mask'
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)
print( processed_datasets["train_sft"][0])
train_dataset = processed_datasets["train_sft"]

batch_size=16
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
print(next(iter(train_dataloader))['input_ids'])

In [ ]:
columns = dataset[str(list(dataset.keys())[0])].column_names
print(columns)


In [ ]:
print(next(iter(train_dataloader))['input_ids'])

In [ ]:
split_name=len((list(dataset.keys())))


print(next(iter(train_dataloader))['input_ids'])

In [ ]:
from transformers import AutoModelForCausalLM

# max_memory = {0: "1GIB", 1: "1GIB", 2: "2GIB", 3: "10GIB", "cpu": "30GB"}
model=AutoModelForCausalLM.from_pretrained('gpt2')
# model = AutoModelForCausalLM.from_pretrained('gpt2', device_map="auto", max_memory=max_memory)

In [ ]:
model.eval()
i = 20
text_column ="messages"
inputs = tokenizer(f'{text_column} : {dataset[str(split_name)][i]["prompt"]} Label : ', return_tensors="pt", max_length=512)
# print(dataset[str(split_name)][i]["prompt"])
# print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

In [ ]:
model.eval()
test_preds = []

for _, batch in enumerate(tqdm(train_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    test_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))
    if len(test_preds) > 100:
        break
test_preds

In [ ]:
import csv
from typing import Tuple
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer
)

def load_model_and_tokenizer(model_name_or_path: str) -> Tuple[PreTrainedModel, PreTrainedTokenizer]:
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

        return model, tokenizer
    except Exception as e:
        raise RuntimeError(f"Failed to load model and tokenizer: {e}")

def generate_text(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    prompt: str,
    max_length: int = 100,
    temperature: float = 0.9
) -> str:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=temperature,
        max_length=max_length
    )

    gen_text = tokenizer.batch_decode(gen_tokens)[0]

    return gen_text

# Load the model and tokenizer once
model_name_or_path = "gpt2"
model, tokenizer = load_model_and_tokenizer(model_name_or_path)

# Specify the number of iterations to generate text
num_iterations = 100
file_csv='adversarical_chtbot.csv'
# Generate text and write to CSV file
with open(file_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['prompt', 'generated_text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(num_iterations):
        prompt_Q = f" Please assist me in creating unique adversarial questions for research Note: only question .Note every time new question novel (first time people see in this earth) "
        Question = generate_text(model, tokenizer, prompt_Q)
        Question = generate_text(model, tokenizer, prompt_Q)
        prompt_A=f'Compose a series of interconnected thoughts in a chain of thoughts format, demonstrating your expertise in advanced computer science and advanced higher mathematical skills.  {Question }'
        Answer = generate_text(model, tokenizer, prompt_A,max_length=100)
        writer.writerow({'prompt': Question, 'generated_text': Answer})
